In [1]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: streamlit in c:\users\hp\appdata\local\programs\python\python312\lib\site-packages (1.39.0)




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Instalamos la libreria de PLOTLY
%pip install plotly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Instalamos librería para análisis de variables categóricas
%pip install funpymodeling

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install streamlit plotly pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
! npm install localtunnel


up to date, audited 23 packages in 5s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [6]:
pip install panel


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: panel in c:\users\hp\appdata\local\programs\python\python312\lib\site-packages (1.5.2)




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install dash



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

C:\Users\HP\AppData\Local\Temp\ipykernel_17276\8502699.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\HP\AppData\Local\Temp\ipykernel_17276\8502699.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [22]:
import panel as pn
import pandas as pd
import plotly.express as px

# Extender Panel con soporte para Plotly
pn.extension('plotly')

# Cargar los datos desde tu archivo CSV
df_mexico = pd.read_csv('df_mexico_sin_outliers.csv')

# Limpiar la columna de precios: eliminar símbolos, convertir en numérico y manejar errores
df_mexico['price'] = pd.to_numeric(df_mexico['price'].replace('[\$,]', '', regex=True), errors='coerce')
df_mexico['availability_365'] = pd.to_numeric(df_mexico['availability_365'], errors='coerce')
df_mexico['review_scores_rating'] = pd.to_numeric(df_mexico['review_scores_rating'], errors='coerce')
df_mexico['number_of_reviews'] = pd.to_numeric(df_mexico['number_of_reviews'], errors='coerce')

# Rellenar valores faltantes en columnas numéricas con la mediana o 0
df_mexico['price'].fillna(df_mexico['price'].median(), inplace=True)
df_mexico['availability_365'].fillna(0, inplace=True)
df_mexico['review_scores_rating'].fillna(df_mexico['review_scores_rating'].median(), inplace=True)
df_mexico['number_of_reviews'].fillna(0, inplace=True)

# Widgets para el dashboard
neighbourhood_select = pn.widgets.Select(name='Neighbourhood', options=df_mexico['neighbourhood_cleansed'].unique().tolist())
price_range = pn.widgets.IntRangeSlider(name='Price Range', start=int(df_mexico['price'].min()), end=int(df_mexico['price'].max()), value=(int(df_mexico['price'].min()), int(df_mexico['price'].max())))
availability_slider = pn.widgets.IntSlider(name='Availability (Days per year)', start=0, end=365, value=180)

# Widget de selección de frames con tamaño doble
frames_select = pn.widgets.Select(name='Frames', options=['Frame 1', 'Frame 2', 'Frame 3'], width=800)

# Gráfica de dispersión para el Frame 1
@pn.depends(neighbourhood_select, price_range, availability_slider)
def update_plot(neighbourhood, price_range, availability):
    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood) &
        (df_mexico['price'] >= price_range[0]) &
        (df_mexico['price'] <= price_range[1]) &
        (df_mexico['availability_365'] >= availability)
    ]
    
    fig = px.scatter(
        filtered_df, x='price', y='availability_365', color='review_scores_rating',
        size='accommodates', hover_name='name',
        labels={'price': 'Price (USD)', 'availability_365': 'Availability (Days per year)'},
        title=f'Airbnb Listings in {neighbourhood}',
        color_continuous_scale=['#e4007c', '#ffdd00', '#00b140', '#f58220', '#00aedb']
    )
    
    fig.update_layout(
        autosize=True,
        title_font=dict(size=20),
        title_x=0.5
    )
    
    return fig

# Gráfica de pay (pie chart) para el Frame 2 (Distribución por Tipo de Habitación)
@pn.depends(neighbourhood_select, price_range, availability_slider)
def pie_chart(neighbourhood, price_range, availability):
    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood) &
        (df_mexico['price'] >= price_range[0]) &
        (df_mexico['price'] <= price_range[1]) &
        (df_mexico['availability_365'] >= availability)
    ]
    
    fig = px.pie(filtered_df, values='price', names='room_type', 
                 title="Distribución de Precios por Tipo de Habitación",
                 color_discrete_sequence=['#e4007c', '#ffdd00', '#00b140', '#f58220', '#00aedb'])  # Misma paleta de colores
    return fig

# Gráfico de burbujas para el Frame 3
@pn.depends(neighbourhood_select, price_range, availability_slider)
def bubble_chart(neighbourhood, price_range, availability):
    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood) &
        (df_mexico['price'] >= price_range[0]) &
        (df_mexico['price'] <= price_range[1]) &
        (df_mexico['availability_365'] >= availability)
    ]
    
    fig = px.scatter(filtered_df, x='availability_365', y='price', size='number_of_reviews', color='property_type',
                     title="Precio vs Disponibilidad por Tipo de Propiedad",
                     labels={'price': 'Price (USD)', 'availability_365': 'Availability (Days per year)', 'number_of_reviews': 'Number of Reviews'},
                     color_discrete_sequence=px.colors.qualitative.Bold,  # Colores más contrastantes
                     size_max=60)
    
    fig.update_layout(autosize=True, title_font=dict(size=20), title_x=0.5)
    return fig

# Función para cambiar entre Frames
@pn.depends(frames_select, neighbourhood_select, price_range, availability_slider)
def display_frame(frames, neighbourhood, price_range, availability):
    if frames == 'Frame 1':
        return pn.pane.Plotly(update_plot(neighbourhood, price_range, availability))
    elif frames == 'Frame 2':
        return pn.pane.Plotly(pie_chart(neighbourhood, price_range, availability))
    elif frames == 'Frame 3':
        return pn.pane.Plotly(bubble_chart(neighbourhood, price_range, availability))

# Sidebar con encabezados y selectbox para Frames
sidebar = pn.Column(
    pn.pane.Markdown("### Panel de selección"),
    neighbourhood_select,
    price_range,
    availability_slider
)

# Layout del dashboard con título y menú de Frames en la parte superior derecha
dashboard = pn.Column(
    pn.Row(
        pn.pane.Markdown("""
        <div style="background-color:#e4007c; width:100%; display:flex; justify-content:center; padding: 10px;">
            <h1 style="font-size: 36px; color: white;">Airbnb Analysis Dashboard</h1>
        </div>
        """),
        frames_select  # Widget de selección más grande (ancho 800)
    ),
    pn.Row(
        sidebar,
        pn.Column(
            display_frame  # Cambia el contenido basado en el frame seleccionado
        )
    )
)

# Iniciar el servidor de Panel
if __name__ == '__main__':
    pn.serve(dashboard)


<>:12: SyntaxWarning:

invalid escape sequence '\$'

<>:12: SyntaxWarning:

invalid escape sequence '\$'

C:\Users\HP\AppData\Local\Temp\ipykernel_13228\2259347151.py:12: SyntaxWarning:

invalid escape sequence '\$'



C:\Users\HP\AppData\Local\Temp\ipykernel_13228\2259347151.py:18: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\HP\AppData\Local\Temp\ipykernel_13228\2259347151.py:19: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

Launching server at http://localhost:63466


2024-10-17 23:12:39.584 200 GET / (::1) 250.57ms
2024-10-17 23:12:39.978 200 GET /static/extensions/panel/bundled/plotlyplot/mapbox-gl-js/v3.0.1/mapbox-gl.css?v=1.5.2 (::1) 3.00ms
2024-10-17 23:12:39.988 200 GET /static/extensions/panel/bundled/jquery/jquery.slim.min.js (::1) 4.00ms
2024-10-17 23:12:39.996 200 GET /static/extensions/panel/bundled/reactiveesm/es-module-shims@%5E1.10.0/dist/es-module-shims.min.js (::1) 4.00ms
2024-10-17 23:12:40.050 200 GET /static/js/bokeh.min.js?v=de958ab1870cd066dba119c34f40c5e1ca4af9c0d31988f4020e6c49a056838b8968f69e29f359fb6dae256aed026344b1797302c230126c662521f0895deb32 (::1) 43.17ms
2024-10-17 23:12:40.073 200 GET /static/js/bokeh-gl.min.js?v=674dcea77483f8996a069d63e07657e6f744a37d10997e93f172179a5ba5d1c9d7621815eeb5bd909aa463644476c1382f6254b9f387002e9ccfa6e640ddf0bb (::1) 64.00ms
2024-10-17 23:12:40.129 200 GET /static/js/bokeh-widgets.min.js?v=fbdc924ac6c91fc9327f170e493da343090822b9951b2eaec18e0a2a4811fc5960912ede4066d269976232f7acfcc5b21a36b

In [31]:
import panel as pn
import pandas as pd
import plotly.express as px

# Extender Panel con soporte para Plotly
pn.extension('plotly')

# Cargar los datos desde tu archivo CSV
df_mexico = pd.read_csv('df_mexico_sin_outliers.csv')

# Limpiar la columna de precios: eliminar símbolos, convertir en numérico y manejar errores
df_mexico['price'] = pd.to_numeric(df_mexico['price'].replace('[\$,]', '', regex=True), errors='coerce')
df_mexico['availability_365'] = pd.to_numeric(df_mexico['availability_365'], errors='coerce')
df_mexico['review_scores_rating'] = pd.to_numeric(df_mexico['review_scores_rating'], errors='coerce')

# Rellenar valores faltantes en columnas numéricas con la mediana o 0
df_mexico['price'].fillna(df_mexico['price'].median(), inplace=True)
df_mexico['availability_365'].fillna(0, inplace=True)
df_mexico['review_scores_rating'].fillna(df_mexico['review_scores_rating'].median(), inplace=True)

# Widgets para el dashboard
neighbourhood_select = pn.widgets.Select(name='Neighbourhood', options=['Promedio'] + df_mexico['neighbourhood_cleansed'].unique().tolist())
price_range = pn.widgets.IntRangeSlider(name='Price Range', start=int(df_mexico['price'].min()), end=int(df_mexico['price'].max()), value=(int(df_mexico['price'].min()), int(df_mexico['price'].max())))
availability_slider = pn.widgets.IntSlider(name='Availability (Days per year)', start=0, end=365, value=180)

# Widget 1: Selectbox más grande
frames_select = pn.widgets.Select(name='Frames', options=['Scatter Plot', 'Pie Chart', 'Bubble Chart', 'Linear Regression Analysis', 'Room Type Insights'], width=400)

# Frame 1: Scatter plot de precio y disponibilidad
@pn.depends(neighbourhood_select, price_range, availability_slider)
def scatter_plot(neighbourhood, price_range, availability):
    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood) & 
        (df_mexico['price'] >= price_range[0]) & 
        (df_mexico['price'] <= price_range[1]) & 
        (df_mexico['availability_365'] >= availability)
    ]
    
    fig = px.scatter(
        filtered_df, x='price', y='availability_365', color='review_scores_rating',
        size='accommodates', hover_name='property_type',
        labels={'price': 'Price (USD)', 'availability_365': 'Availability (Days per year)'},
        title=f'Airbnb Listings in {neighbourhood}',
        color_continuous_scale=['#e4007c', '#ffdd00', '#00b140', '#f58220', '#00aedb']
    )
    fig.update_layout(
        autosize=True,
        title_font=dict(size=20),
        title_x=0.5
    )
    return fig

# Frame 2: Pie chart (Distribución por tipo de habitación)
@pn.depends(neighbourhood_select, price_range, availability_slider)
def pie_chart(neighbourhood, price_range, availability):
    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood) & 
        (df_mexico['price'] >= price_range[0]) & 
        (df_mexico['price'] <= price_range[1]) & 
        (df_mexico['availability_365'] >= availability)
    ]
    
    fig = px.pie(filtered_df, values='price', names='room_type', 
                 title="Distribución de Precios por Tipo de Habitación",
                 color_discrete_sequence=['#e4007c', '#ffdd00', '#00b140', '#f58220', '#00aedb'])
    return fig

# Frame 3: Bubble chart (Precio vs Disponibilidad)
@pn.depends(neighbourhood_select, price_range, availability_slider)
def bubble_chart(neighbourhood, price_range, availability):
    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood) & 
        (df_mexico['price'] >= price_range[0]) & 
        (df_mexico['price'] <= price_range[1]) & 
        (df_mexico['availability_365'] >= availability)
    ]
    
    fig = px.scatter(filtered_df, x='availability_365', y='price', color='property_type', size='review_scores_rating',
                     title="Precio vs Disponibilidad por Tipo de Propiedad",
                     labels={'price': 'Price (USD)', 'availability_365': 'Availability (Days per year)'},
                     color_discrete_sequence=['#e4007c', '#ffdd00', '#00b140', '#f58220', '#00aedb'])
    fig.update_layout(autosize=True, title_font=dict(size=20), title_x=0.5)
    return fig

# Frame 4: Análisis de Regresión Lineal (Price - review_scores_rating)
@pn.depends(neighbourhood_select, price_range, availability_slider)
def linear_regression(neighbourhood, price_range, availability):
    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood) & 
        (df_mexico['price'] >= price_range[0]) & 
        (df_mexico['price'] <= price_range[1]) & 
        (df_mexico['availability_365'] >= availability)
    ]
    
    fig = px.scatter(filtered_df, x='price', y='review_scores_rating', trendline="ols", 
                     title="Regresión Lineal: Precio vs Calificación",
                     labels={'price': 'Price (USD)', 'review_scores_rating': 'Review Score'},
                     color_discrete_sequence=['#e4007c', '#ffdd00', '#00b140', '#f58220', '#00aedb'])
    fig.update_layout(autosize=True, title_font=dict(size=20), title_x=0.5)
    return fig

# Frame 5: Mejor Room Type en promedio
@pn.depends(neighbourhood_select, price_range, availability_slider)
def room_type_insights(neighbourhood, price_range, availability):
    if neighbourhood == 'Promedio':
        grouped_df = df_mexico.groupby('room_type').agg({'review_scores_rating': 'mean'}).reset_index()
        fig = px.bar(grouped_df, x='room_type', y='review_scores_rating', color='room_type',
                     title="Mejor Room Type en Promedio",
                     labels={'room_type': 'Tipo de Habitación', 'review_scores_rating': 'Rentabilidad (Score)'},
                     color_discrete_sequence=['#e4007c', '#ffdd00', '#00b140'])
        return fig

    filtered_df = df_mexico[
        (df_mexico['neighbourhood_cleansed'] == neighbourhood)
    ]
    grouped_df = filtered_df.groupby('room_type').agg({'review_scores_rating': 'mean'}).reset_index()
    
    fig = px.bar(grouped_df, x='room_type', y='review_scores_rating', color='room_type',
                 title=f"Mejor Room Type en {neighbourhood}",
                 labels={'room_type': 'Tipo de Habitación', 'review_scores_rating': 'Rentabilidad (Score)'},
                 color_discrete_sequence=['#e4007c', '#ffdd00', '#00b140'])
    return fig

# Función para cambiar entre Frames
@pn.depends(frames_select, neighbourhood_select, price_range, availability_slider)
def display_frame(frames, neighbourhood, price_range, availability):
    if frames == 'Scatter Plot':
        return pn.pane.Plotly(scatter_plot(neighbourhood, price_range, availability))
    elif frames == 'Pie Chart':
        return pn.pane.Plotly(pie_chart(neighbourhood, price_range, availability))
    elif frames == 'Bubble Chart':
        return pn.pane.Plotly(bubble_chart(neighbourhood, price_range, availability))
    elif frames == 'Linear Regression Analysis':
        return pn.pane.Plotly(linear_regression(neighbourhood, price_range, availability))
    elif frames == 'Room Type Insights':
        return pn.pane.Plotly(room_type_insights(neighbourhood, price_range, availability))

# Sidebar con encabezados y selectbox para Frames
sidebar = pn.Column(
    pn.pane.Markdown("### Panel de selección"),
    neighbourhood_select,
    price_range,
    availability_slider
)

# Layout del dashboard con título y menú de Frames en la parte superior derecha
dashboard = pn.Column(
    pn.Row(
        pn.pane.Markdown("""
        <div style="background-color:#e4007c; width:100%; display:flex; justify-content:center; padding: 10px;">
            <h1 style="font-size: 36px; color: white;">Airbnb Analysis Dashboard</h1>
        </div>
        """),
        frames_select  # Widget de selección en la parte superior derecha
    ),
    pn.Row(
        sidebar,
        pn.Column(
            display_frame  # Cambia el contenido basado en el frame seleccionado
        )
    )
)

# Iniciar el
# Iniciar el servidor de Panel
if __name__ == '__main__':
    pn.serve(dashboard)


<>:12: SyntaxWarning:

invalid escape sequence '\$'

<>:12: SyntaxWarning:

invalid escape sequence '\$'

C:\Users\HP\AppData\Local\Temp\ipykernel_13228\316526557.py:12: SyntaxWarning:

invalid escape sequence '\$'



C:\Users\HP\AppData\Local\Temp\ipykernel_13228\316526557.py:17: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\HP\AppData\Local\Temp\ipykernel_13228\316526557.py:18: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

Launching server at http://localhost:54107


2024-10-18 20:35:58.708 200 GET / (::1) 446.85ms
2024-10-18 20:35:59.072 200 GET /static/extensions/panel/bundled/plotlyplot/mapbox-gl-js/v3.0.1/mapbox-gl.css?v=1.5.2 (::1) 225.97ms
2024-10-18 20:35:59.851 200 GET /static/js/bokeh-widgets.min.js?v=fbdc924ac6c91fc9327f170e493da343090822b9951b2eaec18e0a2a4811fc5960912ede4066d269976232f7acfcc5b21a36b40b14d357c113cad957eb9cb3a7 (::1) 750.81ms
2024-10-18 20:36:00.401 200 GET /static/extensions/panel/bundled/reactiveesm/es-module-shims@%5E1.10.0/dist/es-module-shims.min.js (::1) 1300.96ms
2024-10-18 20:36:00.813 200 GET /static/extensions/panel/bundled/jquery/jquery.slim.min.js (::1) 1713.42ms
2024-10-18 20:36:01.847 200 GET /static/extensions/panel/bundled/plotlyplot/plotly-2.31.1.min.js (::1) 2747.23ms
2024-10-18 20:36:02.470 200 GET /static/js/bokeh.min.js?v=de958ab1870cd066dba119c34f40c5e1ca4af9c0d31988f4020e6c49a056838b8968f69e29f359fb6dae256aed026344b1797302c230126c662521f0895deb32 (::1) 3370.41ms
2024-10-18 20:36:02.741 200 GET /stati

2024-10-18 20:36:16,862 ERROR: panel.reactive - Callback failed for object named 'Frames' changing property {'value': 'Linear Regression Analysis'} 
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\panel\reactive.py", line 462, in _process_events
    self.param.update(**self_params)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2319, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2352, in _update
    self_._batch_call_watchers()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\parameterized.py", line 2546, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\param\paramet

2024-10-18 20:36:17.108 Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001FBC713A270>>, <Task finished name='Task-86602' coro=<ServerSession.with_document_locked() done, defined at c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py:77> exception=ModuleNotFoundError("No module named 'statsmodels'")>)
Traceback (most recent call last):
  File "C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 750, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 774, in _discard_future_result
    future.result()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\bokeh\server\session.py", line 98, in _needs_document_lock_wrapper
    result = await result
             ^^^^^^^^^^^^
  File "c:\Users\HP\AppData\